#### 
### <font color="GREEN">Introduction to Web Scraping</font>
# <font color="RED">Amazon Reviews Scraping</font>

In [13]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [14]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [15]:
# URL of The amazon Review page
reviews_url = 'https://www.amazon.com/Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/'

In [16]:
# Define Page No
len_page = 4

### <font color="red">Functions</font>

In [17]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [18]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

### <font color="red">Data Process</font>

In [19]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [20]:
# Empty List to Hold all reviews data
reviews = []

In [21]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [22]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [23]:
df_reviews

,Name,Stars,Title,Date,Description
0,Jeff,4.0,"Rugged, good look and nice fit, but should you...",02/10/2018,The short answer to if you should go down from...
1,M. Burke,5.0,My Favorite Jacket,19/04/2023,I went through the most recent winter with thi...
2,Jake,4.0,Decent shirt jacket,27/04/2023,I’ve been wearing this regularly for about a m...
3,joe j,4.0,Not bad. Its a good light jacket of sorts,05/04/2023,Its not really a jacket and not really a overs...
4,Jorge Risco,5.0,"Muy buen material, abrigados y queda con todo",24/05/2023,"Muy buen material, para mi que vivo en el sur ..."
5,Christopher Baca,4.0,Gentle Pockets,01/05/2023,The pocket lining is a very gentle polyester-e...
6,Dalton J.,4.0,Fits great just a little lose around the chest,26/05/2023,Worth the money
7,Dante Sparda,5.0,Looks just like the picture,29/05/2023,"light weight, comfortable and looks good. keep..."
8,Dave Warrick,5.0,Absolutely love this jacket!,21/04/2023,After initially seeing this jacket and falling...
9,Megan&Jordan,5.0,fits true to size,31/05/2023,this jacket was very comfy and warm perfect fo...


In [26]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)